In [43]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import json
import os

path = r'C:\Users\ruanv\OneDrive\Documentos\Mestrado\Projeto de Mestrado\bases'

df = pd.read_csv(os.path.join(path,'demandaTrafos_2018-2023.csv'), sep=';', encoding='latin-1')

data = df

In [44]:
df = data

In [45]:
df = df[(df['Potencia Ativa'] > 0) & (df['Potencia Ativa'] < 30)]
df = df[df['Potencia Ativa'].notna()]

df['Data/Hora Medição'] = pd.to_datetime(df['Data/Hora Medição'], dayfirst=True, errors='coerce')

df['Dia'] = df['Data/Hora Medição'].dt.date

df = df.groupby(['TRAFO', 'Dia'])[['Potencia Ativa', 'Potencia Reativa']].mean().reset_index()

# Calculando a Potência Aparente e o Fator de Potência
df['Potencia Aparente'] = np.sqrt(df['Potencia Ativa']**2 + df['Potencia Reativa']**2)
df['Fator de Potência'] = np.where(df['Potencia Aparente'] != 0, df['Potencia Ativa'] / df['Potencia Aparente'], np.nan)

In [46]:
df = df[['TRAFO', 'Dia', 'Potencia Aparente','Fator de Potência']]

In [47]:
data2 = df[df['TRAFO'] == 'CTR-02T1'] # Pra filtrar um so trafo. na pratica to plotando tudo mesmo e seleciono no plot

In [ ]:
df['Dia'] = pd.to_datetime(df['Dia'])

# Potencia Aparente ao longo do tempo, separada por TRAFO (dois cliques marca/desmarca todos)
fig_aparente = px.line(df, x='Dia', y='Potencia Aparente', color='TRAFO', 
                       title='Potência Aparente ao Longo do Tempo por Transformador',
                       labels={'Potencia Aparente': 'Potência Aparente (kVA)', 'Dia': 'Data'})

fig_aparente.show()
